# Make token vocabulary
Train a tokenizer on the Tira ASR dataset

In [1]:
from tokenizers import (
    decoders,
    models,
    normalizers,
    pre_tokenizers,
    processors,
    trainers,
    Tokenizer,
)
from transformers import WhisperTokenizer
import pandas as pd
import numpy as np
import seaborn
import string
from clap.encoders import *
from transformers import DebertaV2Tokenizer, AutoProcessor
import torch
from scipy.spatial.distance import cdist, cosine, squareform
from tqdm import tqdm
tqdm.pandas()
from unidecode import unidecode
import random
import json

/usr/local/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
metadata='/Users/markjos/projects/malachor5/data/hf-datasets/tira-clean/metadata.csv'

In [5]:
df=pd.read_csv(metadata)
def get_text_corpus():
    train_df=df[df['split']=='train']
    for row in train_df['transcription']:
        yield row

In [6]:
unique_words = set()
for text in get_text_corpus():
    unique_words.update(text.split())
len(unique_words)

7246

In [22]:
unk = "[UNK]"
trainer=trainers.BpeTrainer(vocab_size=7000, special_tokens=[unk], max_token_length=8, min_frequency=2)

In [23]:
tokenizer = Tokenizer(models.BPE())
tokenizer.train_from_iterator(get_text_corpus(), trainer=trainer)
tira_vocab=tokenizer.get_vocab()
len(tira_vocab)

5810

In [29]:
i=random.randint(0,len(df))
line=df['transcription'].iloc[i]
encoding=tokenizer.encode(line)

line, encoding.tokens, encoding.ids

('bàcʊ́m ŋgɔ́t̪ɔ̀ ŋùɽùvɔ́ ŋàmɽàrɛ̀',
 ['bàcʊ́', 'm ŋgɔ́', 't̪ɔ̀ ', 'ŋùɽù', 'vɔ́ ', 'ŋàmɽà', 'rɛ̀'],
 [677, 1725, 229, 2238, 1564, 684, 122])

In [24]:
vocab_lens=[]
for item in tira_vocab:
    vocab_lens.append(len(item.strip().replace('#', '')))
np.array(vocab_lens).mean()

5.174526678141136